In [15]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tensorflow as tf

# Generate synthetic data
num_samples = 1000
timesteps = 10
input_dim = 1

np.random.seed(0)
X = np.random.rand(num_samples, timesteps, input_dim)
y_NDBC = np.random.rand(num_samples, 1)
y_ERA5 = np.random.rand(num_samples, 1)

# Your custom loss function
def combined_mse(alpha=0.5):
    def loss(y_true, y_pred):
        y_true_1, y_true_2 = tf.split(y_true, num_or_size_splits=2, axis=1)

        mse_1 = tf.reduce_mean(tf.square(y_pred - y_true_1))
        mse_2 = tf.reduce_mean(tf.square(y_pred - y_true_2))
        return alpha * mse_1 + (1 - alpha) * mse_2
    return loss

# Define your LSTM model using the Sequential API
neurons = 50
batch_size = 25

model = Sequential()
model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
model.add(Dense(1))

# Compile the model with the custom loss function
y_combined = np.concatenate([y_NDBC, y_ERA5], axis=1)
model.compile(optimizer='adam', loss=combined_mse(alpha=0.5))

# Split the data into training and validation sets (80% training, 20% validation)
split_index = int(num_samples * 0.8)
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y_combined[:split_index], y_combined[split_index:]

# Train your model
epochs = 10
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)

Epoch 1/10
32/32 [==============================] - 1s 7ms/step - loss: 0.1097 - val_loss: 0.0874
Epoch 2/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0863 - val_loss: 0.0830
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 0.0854 - val_loss: 0.0820
Epoch 4/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0849 - val_loss: 0.0817
Epoch 5/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0858 - val_loss: 0.0814
Epoch 6/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0847 - val_loss: 0.0820
Epoch 7/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0850 - val_loss: 0.0817
Epoch 8/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0842 - val_loss: 0.0811
Epoch 9/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0838 - val_loss: 0.0813
Epoch 10/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0839 - val_loss: 0.0806


In [16]:
model.predict(10, batch_size=1)

IndexError: tuple index out of range

# Austins Code:


In [ ]:
def penalized_loss_fn(lam):
    def loss(y_true, y_pred):

        """The y_true term contains the regularizor HyCOM step on the second axis.
        This is not directly generated by the model, so we slice the term off the end
        and simply use it as a regularization term to the buoy forecast.

        lam = 0 is no real buoy data used, all HyCOM regularization
        lam = 1 is all real buoy data, no HyCOM regularization"""

        print(y_true.shape, y_pred.shape)

        water_temp_true = y_true[:,0]
        hy_water_temp_true = y_true[:,3]

        water_delta1 = y_pred[:,0] - water_temp_true
        water_delta2 = y_pred[:,0] - hy_water_temp_true

        gust_temp_true = y_true[:,1]
        c_gust_temp_true = y_true[:,11]

        gust_delta1 = y_pred[:,1] - gust_temp_true
        gust_delta2 = y_pred[:,1] - c_gust_temp_true

        pres_temp_true = y_true[:,2]
        c_pres_temp_true = y_true[:,15]

        pres_delta1 = y_pred[:,2] - pres_temp_true
        pres_delta2 = y_pred[:,2] - c_pres_temp_true


        model_pred = tf.math.reduce_mean(keras.backend.abs(y_pred[:,3:]-y_true[:,3:]))

        temp_out =  tf.math.reduce_mean( tf.math.scalar_mul(lam, keras.backend.abs(water_delta1))
                                  + tf.math.scalar_mul((1-lam), keras.backend.abs(water_delta2)))

        gust_out = tf.math.reduce_mean( tf.math.scalar_mul(lam, keras.backend.abs(gust_delta1))
                                  + tf.math.scalar_mul((1-lam), keras.backend.abs(gust_delta2)))

        pres_out = tf.math.reduce_mean( tf.math.scalar_mul(lam, keras.backend.abs(pres_delta1))
                                  + tf.math.scalar_mul((1-lam), keras.backend.abs(pres_delta2)))

        return temp_out+gust_out+pres_out+model_pred

    return loss

def build_1D_gru(lam, window_size, num_channels, deep_layers):

    input = layers.Input(shape=(num_channels, window_size))

    x = layers.Reshape(target_shape=(window_size,num_channels))(input)
    x = layers.Dense(256, activation='tanh')(x)
    x = layers.BatchNormalization(scale=True, center=True, momentum=0.999) (x)
    x = layers.Dropout(0.1)(x)
    x = layers.GRU(256, activation='tanh', return_sequences=True)(x)
    x = layers.Dropout(0.1)(x)
    x = layers.GRU(256, activation='tanh', return_sequences=True)(x)


    for d in range(deep_layers):
        x = layers.Dense(256, activation='tanh')(x)
        x = layers.BatchNormalization(scale=True, center=True, momentum=0.999) (x)
        x = layers.Dropout(0.1)(x)
        x = layers.GRU(256, activation='tanh', return_sequences=True)(x)
        x = layers.Dropout(0.1)(x)

        if (d == deep_layers-1):
            x = layers.GRU(256, activation='tanh', return_sequences=False)(x)

        else:
            x = layers.GRU(256, activation='tanh', return_sequences=True)(x)

    x = layers.Dropout(0.1)(x)
    x = layers.Dense(200, activation='tanh')(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(200, activation='tanh')(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(200, activation='tanh')(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(200, activation='tanh')(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(num_channels, activation='linear')(x)

    model = Model(input, x)
    model.compile(optimizer="rmsprop", loss=[penalized_loss_fn(lam)])

    print(model.summary())

    return model

def train_model(model, x_train, y_train, x_val, y_val):

    history = model.fit(x_train, y_train,
                    epochs=EPOCHS,
                    batch_size=BATCH,
                    shuffle=False,
                    validation_data=(x_val, y_val),)# verbose=0)

    return history


def display_history(history, model, x_test, y_test):
    #res = model.evaluate(x=x_test,y=y_test)
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'Model Evaluated Training Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'])
    plt.show()

def get_predictions(model, x_test):
    predictions = model.predict(x=x_test,verbose = 0)
    return predictions

def get_multi_predictions(model, x_test, horizon_size):
    predictions=[]
    for i in range(horizon_size):
        x_test = model.predict(x=x_test[:,1:],verbose = 0)
        predictions.append(x_test)
    return np.array(predictions)